# Générer les sites synthétiques (MLP et harmonisation)

Ce notebook produit des sites synthétiques à partir des compilations MLP et harmonisées en utilisant `robust_evaluation_tools.synthectic_sites_generations.generate_sites`.
Chaque configuration crée 12 dossiers (3 tailles d'échantillon × 2 ratios de malades × 2 répétitions).


In [ ]:
# Imports et configuration
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path('.').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Décommente si les dépendances manquent
# !pip install joblib scikit-learn

from robust_evaluation_tools.synthectic_sites_generations import generate_sites

try:
    import joblib  # noqa: F401
    import sklearn  # noqa: F401
except ImportError as exc:
    raise ImportError("Installe joblib et scikit-learn (pip install joblib scikit-learn)") from exc


In [ ]:
# Paramètres communs (12 dossiers par dataset)
sample_sizes = [100]  # 1 taille
disease_ratios = [0.03, 0.1, 0.3, 0.5, 0.7, 0.8]    # 2 ratios (0% et 50% malades)
num_tests = 40                  # 2 répétitions => 3*2*2 = 12 dossiers
synthetic_version = 'v1'       # 'v1' ou 'v2' selon le pipeline
disease = 'ALL'           # 'ALL', 'ASTMIX' ou une maladie précise (ex: 'AD')
fixed_bias = False
n_jobs = -1                     # -1 pour tous les coeurs

camcan_hc_only = True         # Filtrer les HC hors CamCAN
include_camcan = True        # Inclure les compilations avec CamCAN
compilation_suffix = '.with_camcan' if include_camcan else ''

# Choix de l'augmentation (None pour utiliser les données originales, ou un entier -> *_AUG_{n})
augmentation_copies = 5
augmentation_suffix = f"_AUG_{augmentation_copies}" if augmentation_copies else ''

def build_dataset_config(base_name: str):
    input_dir = Path(f"DONNES/processed/compilation/classic/{base_name}{augmentation_suffix}")
    return {
        'name': f"{base_name}{augmentation_suffix}",
        'data_path': input_dir / f"compilation.all_metrics{compilation_suffix}.csv.gz",
        'output_dir': Path(f"DONNES/processed/synthetic_sites/{base_name}{augmentation_suffix}/{disease}"),
    }

datasets = [
    build_dataset_config('mlp'),
    build_dataset_config('harmonized'),
]


In [ ]:
# Aperçu des datasets (premières lignes)
for cfg in datasets:
    print(f"Dataset: {cfg['name']} -> {cfg['data_path']}")
    display(pd.read_csv(cfg['data_path']).head(3))
    print('-'*60)


In [ ]:
# Génération des sites synthétiques pour chaque dataset
for cfg in datasets:
    cfg['output_dir'].mkdir(parents=True, exist_ok=True)
    print(f"\nGénération pour {cfg['name']} -> {cfg['output_dir']}\n")
    generate_sites(
        sample_sizes=sample_sizes,
        disease_ratios=disease_ratios,
        num_tests=num_tests,
        directory=str(cfg['output_dir']),
        data_path=str(cfg['data_path']),
        SYNTHETIC_SITES_VERSION=synthetic_version,
        disease=disease,
        camcan_hc_only=camcan_hc_only,
        fixed_biais=fixed_bias,
        n_jobs=n_jobs,
    )
    print(f"Terminé : {cfg['output_dir'].resolve()}")


In [ ]:
# Résumé des dossiers et fichiers générés
from collections import Counter

for cfg in datasets:
    generated_dirs = sorted([p for p in cfg['output_dir'].glob('*') if p.is_dir()])
    print(f"\n{cfg['name']} -> {len(generated_dirs)} dossiers générés (attendu: 12)")
    for d in generated_dirs:
        csv_count = len(list(d.rglob('*.csv')))
        print(f"  {d.relative_to(PROJECT_ROOT)} : {csv_count} fichiers csv")
    sample_csvs = sorted(cfg['output_dir'].rglob('*.csv'))
    if sample_csvs:
        print(f"Aperçu du premier fichier pour {cfg['name']}: {sample_csvs[0]}")
        display(pd.read_csv(sample_csvs[0]).head())
